In [ ]:
import math
import os

import cv2
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

from config import Config
from datagen import DataGenerator, _relative_points, _pad_img
from model import SphinxModel
%matplotlib inline

%load_ext autoreload
%autoreload 2

## 初始化数据集和模型

#### 注意修改以下值：
1. CUDA_VISIBLE_DEVICES
2. cfg.load
3. generate_set的模式

#### 关于初始化和硬件占用
1. 如果需要从验证集换到测试集，模型不需要重新初始化。dataset必须重新初始化
2. 模型初始化后，关闭该页面不会终止显存占用，再次打开页面也不需要重新初始化
3. 如果要释放显存，需要在页面内restart kernel，或在jupyter首页shutdown该页面

In [ ]:
cfg = Config()

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
cfg.load = "sphinx_14"

dataset = DataGenerator(cfg)
dataset.generate_set(train = True)

In [ ]:
model = SphinxModel(cfg, dataset)
model.generate_model(False)

In [ ]:
with tf.name_scope('Session'):
    model.Session = tf.Session()
    model._init_variable()

## 在训练集和验证集上进行测试

#### 每次只测试1张图片。修改i控制图片位置
#### 修改dataset.{valid_set, train_set} 控制训练集或验证集

In [ ]:
i = 0

In [ ]:
# get image info
name = dataset.valid_set[i]
point = dataset.data_dict[name]['points']
label = dataset.data_dict[name]['label']
weight = np.asarray(dataset.data_dict[name]['weight'])

# open image
img = dataset.open_img(dataset.train_img_dir, name)
orig_size = max(img.shape)

# pad image and points
new_p = _relative_points(point, img.shape)
img = _pad_img(img)

# visualize ground truth
gt_hm_visual = dataset._generate_hm(orig_size, cfg.img_size, new_p, weight, keep_invisible=False)
gt_hm_visual = np.sum(gt_hm_visual, 2) * 255
gt_hm_visual = cv2.resize(gt_hm_visual, (orig_size, orig_size), interpolation=cv2.INTER_LINEAR)
gt_hm_visual = np.expand_dims(gt_hm_visual, 2)
gt_visual = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) + gt_hm_visual.astype(np.int32)
pred_visual = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# resize image for inference
img = cv2.resize(img, (cfg.img_size, cfg.img_size), interpolation=cv2.INTER_LINEAR)
img = img.astype(np.float32) / 255
images = np.expand_dims(img, 0)

# get prediction
pred = model.Session.run(model.output[2], feed_dict={model.img: images})
pred = pred[0]

# calculate predictions from heatmap
pred_idxes = np.zeros((model.num_points, 2), dtype=np.float32)
total_dist = 0.0
for j in range(model.num_points):
    if weight[j] == 1:
        index = np.unravel_index(pred[:, :, j].argmax(), (model.img_size, model.img_size))
        index = (index[1], index[0])
        pred_idx = np.array(index) / model.img_size * orig_size
        pred_idxes[j, :] = pred_idx
        total_dist += np.linalg.norm(pred_idx - new_p[j])

# visualize prediction
pred_hm_visual = dataset._generate_hm(orig_size, cfg.img_size, pred_idxes, weight, keep_invisible=False)
pred_hm_visual = np.sum(pred_hm_visual, 2) * 255
pred_hm_visual = cv2.resize(pred_hm_visual, (orig_size, orig_size), interpolation=cv2.INTER_LINEAR)
pred_hm_visual = np.expand_dims(pred_hm_visual, 2)
pred_visual = pred_visual + pred_hm_visual.astype(np.int32)        
  
# calc normalized average error
if label <= 2:
    norm_idx1 = new_p[5]
    norm_idx2 = new_p[6]
else:
    norm_idx1 = new_p[15]
    norm_idx2 = new_p[16]

norm_dist = np.linalg.norm(norm_idx2 - norm_idx1)
error = total_dist / norm_dist / np.sum(np.count_nonzero(weight))

# show results
plt.imshow(gt_visual)
plt.show()
plt.imshow(pred_visual)
plt.show()
print("image error:", error)
        
i += 1

## 在测试集上测试

#### 重新初始化了dataset. 使用test_generator进行测试
#### 每次测试一张图片，不能控制测试进度

In [ ]:
import utils as ut
dataset.generate_set(train = False)
test_gen = dataset.test_generator(cfg.img_size, 1)

In [ ]:
# Traversal the test set
images, categories, offsets, names, sizes = next(test_gen)
prediction = model.Session.run(model.output[2], feed_dict={model.img: images})
hms = prediction

# init
pred_idxes = np.zeros((model.num_points, 2), dtype=np.float32)
img = images[0]
hm = hms[0]
offset = offsets[0]
category = categories[0]
name = names[0]
size = sizes[0]

# predict
write_line = [name, category]
for j in range(model.num_points):
    if ut.VALID_POSITION[category][j] is 1:
        index = np.unravel_index(hm[:, :, j].argmax(), (model.img_size, model.img_size))
        index = (index[1], index[0])
        point = np.array(index) / model.img_size * size
        pred_idxes[j] = point
        point -= offset
        write_line.append(str(int(round(point[0]))) + '_' + str(int(round(point[1]))) + '_1')
    else:
        write_line.append('-1_-1_-1')
print(write_line)
    
# visualize prediction
pred_hm_visual = np.zeros((size, size, 3))
pred_hm = dataset._generate_hm(size, size, pred_idxes, ut.VALID_POSITION[category], keep_invisible=False)
pred_hm = np.sum(pred_hm, 2) * 255
pred_hm_visual[:, :, 1] = pred_hm
pred_visual = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (size, size), interpolation=cv2.INTER_LINEAR)
pred_visual = pred_visual + pred_hm_visual.astype(np.int32)
# show results
plt.imshow(pred_visual)
plt.show()